In [6]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import numpy as np

from smt.sampling_methods import LHS

## True Function
## $f(x, y) = (a+c)x^2 + (b+d)y^2 + dx + cy + a + b + \epsilon$
## $-5<= x<= 5$, $-5<= y<= 5$, a,b,c,dは100個ランダムに生成
## $h(x, y, a, b, c, d) =$ 3deg-Poly-Reg


In [21]:
def f(x, y, a, b, c, d):
    return (a+c)*x**2 + (b+d)*y**2 + d*x + c*y + a + b + np.random.randn()

## Data Generation

In [22]:
xlimits = np.array([[0.0, 1.0], [-1.0, 1.0], [-1, 1], [0, 1]])
sampling = LHS(xlimits=xlimits)

num = 100
input_values = sampling(num)

In [23]:
x = np.arange(-5, 5, 0.1)
y = np.arange(-5, 5, 0.1)
xx, yy = np.meshgrid(x, y)

In [66]:
z = np.zeros([100, 100, 100])
points = []
for k, inp in enumerate(input_values):
    a, b, c, d = inp[0], inp[1], inp[2], inp[3]
    for i in range(100):
        for j in range(100):
            z[k, i, j] = f(xx[i, j], yy[i, j], a, b, c, d)
            points.append([xx[i, j], yy[i, j], a, b, c, d])

## Learning W
## $X = (1, x, y, a, b, c, d, x^2, xy, xa, xb, ...)  (1000000 \times m)$
## $Z = (z(-5, 5, a1, b1, c1, d1), )   (1000000 \times 1)$
## $W = (m\times 1)$
## $Z = XW$

In [93]:
# create X
points = np.array(points)

poly = PolynomialFeatures(degree=3)
X = poly.fit_transform(points)

In [94]:
Z = z.flatten()

In [97]:
model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                  ('linear', LinearRegression(fit_intercept=False))])

In [99]:
learned_model = model.fit(points, Z)

In [103]:
learned_model.predict(points[0][None])

array([50.83373867])

In [112]:
def pred(model, a, b, c, d):
    res = []
    for x in np.arange(-5.0, 5.0, 0.1):
        for y in np.arange(-5.0, 5.0, 0.1):
            res.append(model.predict([[x, y, a, b, c, d]]))
    return np.array(res).reshape([100, 100])

In [120]:
import joblib

In [121]:
joblib.dump(learned_model, "model.pkl")

['model.pkl']